In [1]:
import datetime as dt
import matplotlib.pyplot as plt
import heapq
import networkx as nx

import math
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
stop_pairs_within_500m = pd.read_pickle("./stop_pairs_within_500m.pkl")
final_df = pd.read_pickle("./final_df.pkl")

In [3]:
final_df.head(1)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,route_id,stop_name,stop_lat,stop_lon,parent_station,route_desc,arrival_stop_id,departure_stop_id,arrival_time_seconds,departure_time_seconds,travel_time_seconds
444152,1.TA.91-1R-Y-j24-1.1.H,18:57:00,19:01:00,8501120:0:1,2,91-1R-Y-j24-1,Lausanne,46.516775,6.629513,Parent8501120,EXT,8501120:0:1,NaN,68220,68460,NaN


In [4]:
stop_pairs_within_500m['transfer_time_seconds'] = stop_pairs_within_500m['transfer_time_seconds'].round().astype(int)
stop_pairs_within_500m.head(1)

,stop_id1,stop_id2,stop_name1,stop_name2,stop_lat1,stop_lon1,stop_lat2,stop_lon2,parent_station1,parent_station2,distance_meters,transfer_time_minutes,transfer_time_seconds
0,8501075,8591986,Lausanne-Ouchy (lac),"Lausanne, Beau-Rivage",46.505078,6.627608,46.508305,6.627501,Parent8501075,Parent8591986,358.861431,9.177229,551



## 3. Building the Transportation Graph <a class="anchor" id="transportationgraph"></a>
- NetworkX doesn't work in the PySpark kernel. So we need to run it in `%%local`.
    - For that we need all our PySpark dataframes loaded locally.
- **Do we need to process the stop_ids?** 

In [5]:
#final_df_small = final_df.head(500000) # 500k rows runs for > 5 minutes
#final_df_small = final_df.head(400000) # 400k rows takes < 3 seconds
final_df_small = final_df.head(150000)
#final_df_small = final_df.copy()

G = nx.MultiDiGraph()

for index, row in tqdm(final_df_small.iterrows(), total=final_df_small.shape[0], desc='Adding nodes'):
    G.add_node(row['arrival_stop_id'], 
               stop_name=row['stop_name'],
               stop_lat=row['stop_lat'], 
               stop_lon=row['stop_lon'])

# Pre-sort and group the dataframe by 'trip_id' with sorted 'stop_sequence'
sorted_final_df = final_df_small.sort_values(by=['trip_id', 'stop_sequence'])
grouped_by_trip = sorted_final_df.groupby('trip_id')

# Add edges with route description, processing grouped trips
for trip_id, trip_data in tqdm(grouped_by_trip, desc='Processing trips'):
    previous_row = None
    for idx, row in trip_data.iterrows():
        if previous_row is not None:
            weight = (row['arrival_time_seconds'] - previous_row['departure_time_seconds'])
            G.add_edge(previous_row['stop_id'], row['stop_id'], 
                       departure_time_seconds=previous_row['departure_time_seconds'],
                       arrival_time_seconds=row['arrival_time_seconds'], 
                       weight=weight,
                       route_id=row['route_id'],
                       route_desc=row['route_desc'])
        previous_row = row

# Add walking edges
new_edges_to_add = []  # List to hold edges to be added

for _, row in tqdm(stop_pairs_within_500m.iterrows(), total=stop_pairs_within_500m.shape[0], desc='Adding walking edges'):
    stop_id1 = row['stop_id1']
    stop_id2 = row['stop_id2']
    transfer_time = row['transfer_time_seconds']

    if G.has_node(stop_id1) and G.has_node(stop_id2):
        if stop_id2 in G.succ[stop_id1]:  # Check for existing edges from stop_id1 to stop_id2
            for key, edge_attr in dict(G.succ[stop_id1][stop_id2]).items(): 
                departure_time = edge_attr['departure_time_seconds']
                arrival_time = departure_time + transfer_time

                # Check if a walking edge with the same timing already exists
                existing_edges = [d for u, v, d in G.edges((stop_id1, stop_id2), data=True) 
                                  if d.get('route_id') == 'WALKING' and d['departure_time_seconds'] == departure_time]

                if not existing_edges:
                    # Prepare to add new walking edge
                    new_edges_to_add.append((stop_id1, stop_id2, departure_time, arrival_time, transfer_time))

# Now add new edges after all iterations are complete
for stop_id1, stop_id2, departure_time, arrival_time, transfer_time in new_edges_to_add:
    G.add_edge(stop_id1, stop_id2,
               departure_time_seconds=departure_time,
               arrival_time_seconds=arrival_time,
               weight=transfer_time,
               route_id="WALKING",
               route_desc="walk")

Adding walking edges: 100%|██████████| 14448/14448 [00:40<00:00, 355.49it/s]  


In [6]:
print(f"Number of unique nodes: {G.number_of_nodes()}")
print(f"Number of unique edges: {G.number_of_edges()}")

Number of unique nodes: 531
Number of unique edges: 261061


In [7]:
# print one example of a walking edge
for u, v, attr in G.edges(data=True):
    if attr.get('route_desc', '') == 'walk':
        print(f"From {u} to {v} with attributes: {attr}")
        break

From 8592050 to 8591818 with attributes: {'departure_time_seconds': 65040, 'arrival_time_seconds': 65531, 'weight': 491, 'route_id': 'WALKING', 'route_desc': 'walk'}


In [8]:
# Print node and edge attributes
print("Node attributes:", list(G.nodes(data=True))[0])  # Print attributes of the first node
print("Edge attributes:", list(G.edges(data=True))[0])  # Print attributes of the first edge

# Find a pair of nodes with more than one edge
for u, v in G.edges():  # Iterate through each edge
    edge_data = G.get_edge_data(u, v)  # Fetch the data for the edge between u and v
    if len(edge_data) > 1:  # Check if there are multiple edges between u and v
        print(f"Multiple edges between {u} and {v}:")
        for key, data in edge_data.items():  # Iterate through each edge's data
            print(f"Edge Key: {key}, Data: {data}")
        break

Node attributes: ('8501120:0:1', {'stop_name': 'Lausanne', 'stop_lat': 46.516774559699, 'stop_lon': 6.629512898808})
Edge attributes: ('8501120:0:1', '8501121:0:2', {'departure_time_seconds': 66420, 'arrival_time_seconds': 66600, 'weight': 180, 'route_id': '91-3-T-j24-1', 'route_desc': 'R'})
Multiple edges between 8501120:0:1 and 8501121:0:2:
Edge Key: 0, Data: {'departure_time_seconds': 66420, 'arrival_time_seconds': 66600, 'weight': 180, 'route_id': '91-3-T-j24-1', 'route_desc': 'R'}
Edge Key: 1, Data: {'departure_time_seconds': 44820, 'arrival_time_seconds': 45000, 'weight': 180, 'route_id': '91-3-T-j24-1', 'route_desc': 'R'}
Edge Key: 2, Data: {'departure_time_seconds': 26820, 'arrival_time_seconds': 27000, 'weight': 180, 'route_id': '91-3-T-j24-1', 'route_desc': 'R'}


## 4. Finding the Fastest Route <a class="anchor" id="findingfastestroute"></a>

Reverse Graph
- Start from stop_id_b (destination stop) and search backwards
- For each neighbor node, expand only the latest allowed (departure time >= arrival time) edge ignoring all the earlier ones. 

## Searching Backwards

In [10]:
def find_routes(G, stop_id_a, stop_id_b, latest_arrival_time, k):
    # Reverse the graph
    RG = G.reverse()

    for u, v, key, data in list(RG.edges(data=True, keys=True)):
        arrival_time = data['arrival_time_seconds']
        departure_time = data['departure_time_seconds']
        # Adjust conditions to remove the edge if it does not meet the requirements
        if arrival_time > latest_arrival_time or departure_time + 3600 < latest_arrival_time:
            RG.remove_edge(u, v, key)

    # Priority queue: (negative latest possible departure, node, path, path details)
    pq = [(-latest_arrival_time, stop_id_b, [], [])]
    heapq.heapify(pq)

    # To store found routes
    routes = []

    # While there are nodes to process and we need more routes
    while pq and len(routes) < k:
        curr_time, curr_node, path, path_details = heapq.heappop(pq)
        curr_time = -curr_time  # Convert back to positive time

        # If the current node is the start node and valid route
        if curr_node == stop_id_a:
            # Include stop name and last departure time (if it's the start node, we set it to None)
            stop_info = G.nodes[curr_node].get('stop_name', 'Unknown')
            path_details.append((curr_node, stop_info, None))
            routes.append((curr_time, list(reversed(path_details))))
            continue

        # Iterate over predecessor nodes
        for pred, data in RG[curr_node].items():
            # For each edge from pred to curr_node, find the latest valid edge
            max_depart_time = 0
            edge_attrs = None
            for key, attrs in data.items():
                if attrs['arrival_time_seconds'] <= curr_time:
                    if attrs['departure_time_seconds'] > max_depart_time:
                        max_depart_time = attrs['departure_time_seconds']
                        edge_attrs = attrs

            if max_depart_time > 0:
                # Include additional information about the stop and its departure time
                stop_info = G.nodes[curr_node].get('stop_name', 'Unknown')
                new_path_details = path_details + [(curr_node, stop_info, edge_attrs['departure_time_seconds'])]
                # Push to the priority queue with negative time for max heap effect
                heapq.heappush(pq, (-max_depart_time, pred, path + [curr_node], new_path_details))

    # Sort routes by the latest departure time
    routes.sort(reverse=True, key=lambda x: x[0])
    return routes[:k]

stop_id_a = '8592050' # 'Lausanne, gare'
#stop_id_b = '8591818' # 'Lausanne-Flon, pl. de l'Europe'
#stop_id_b = '8592173' # 'Prilly, centre'
stop_id_b = '8501214' # 'Ecublens VD, EPFL'

latest_arrival_time = 66180 # 18:23:00
k=3
results = find_routes(G, stop_id_a, stop_id_b, latest_arrival_time, k)
results

[(65100,
  [('8592050', 'Lausanne, gare', None),
   ('8591818', "Lausanne-Flon, pl. de l'Europe", 65100),
   ('8592133', 'Lausanne, Vigie', 65400),
   ('8501207', 'Lausanne, Montelly', 65460),
   ('8501208', 'Lausanne, Provence', 65580),
   ('8501209', 'Lausanne, Malley', 65640),
   ('8501210', 'Lausanne, Bourdonnette', 65700),
   ('8501211', 'Chavannes-R., UNIL-Chamberonne', 65880),
   ('8501212', 'Chavannes-R., UNIL-Mouline', 65940),
   ('8501213', 'Ecublens VD, UNIL-Sorge', 66000),
   ('8501214', 'Ecublens VD, EPFL', 66060)]),
 (65040,
  [('8592050', 'Lausanne, gare', None),
   ('8591818', "Lausanne-Flon, pl. de l'Europe", 65040),
   ('8592133', 'Lausanne, Vigie', 65100),
   ('8501207', 'Lausanne, Montelly', 65160),
   ('8501208', 'Lausanne, Provence', 65280),
   ('8501209', 'Lausanne, Malley', 65340),
   ('8501210', 'Lausanne, Bourdonnette', 65400),
   ('8501211', 'Chavannes-R., UNIL-Chamberonne', 65580),
   ('8501210', 'Lausanne, Bourdonnette', 65760),
   ('8501211', 'Chavannes-R.

### The same as ^ but with more output info

In [11]:
def find_routes(G, stop_id_a, stop_id_b, latest_arrival_time, k):
    # Reverse the graph
    RG = G.reverse()

    for u, v, key, data in list(RG.edges(data=True, keys=True)):
        arrival_time = data['arrival_time_seconds']
        departure_time = data['departure_time_seconds']
        # Adjust conditions to remove the edge if it does not meet the requirements
        if arrival_time > latest_arrival_time or departure_time + 3600 < latest_arrival_time:
            RG.remove_edge(u, v, key)
            
    # Priority queue: (negative latest possible departure, node, path, path details)
    pq = [(-latest_arrival_time, stop_id_b, [], [])]
    heapq.heapify(pq)

    # To store found routes
    routes = []

    # While there are nodes to process and we need more routes
    while pq and len(routes) < k:
        curr_time, curr_node, path, path_details = heapq.heappop(pq)
        curr_time = -curr_time  # Convert back to positive time

        # If the current node is the start node and valid route
        if curr_node == stop_id_a:
            # Include stop name and last departure time (if it's the start node, we set it to None)
            stop_info = G.nodes[curr_node].get('stop_name', 'Unknown')
            path_details.append((curr_node, stop_info, None, None, None, None))  # No route info at the starting point
            routes.append((curr_time, list(reversed(path_details))))
            continue

        # Iterate over predecessor nodes
        for pred, data in RG[curr_node].items():
            # For each edge from pred to curr_node, find the latest valid edge
            max_depart_time = 0
            edge_attrs = None
            for key, attrs in data.items():
                if attrs['arrival_time_seconds'] <= curr_time:
                    if attrs['departure_time_seconds'] > max_depart_time:
                        max_depart_time = attrs['departure_time_seconds']
                        edge_attrs = attrs

            if max_depart_time > 0:
                # Include additional information about the stop and its departure time
                stop_info = G.nodes[curr_node].get('stop_name', 'Unknown')
                new_path_details = path_details + [
                    (curr_node, stop_info, edge_attrs['departure_time_seconds'], 
                     edge_attrs['arrival_time_seconds'], edge_attrs.get('route_desc', 'Unknown'), 
                     edge_attrs.get('route_id', 'Unknown'))
                ]
                # Push to the priority queue with negative time for max heap effect
                heapq.heappush(pq, (-max_depart_time, pred, path + [curr_node], new_path_details))

    # Sort routes by the latest departure time
    routes.sort(reverse=True, key=lambda x: x[0])
    return routes[:k]

stop_id_a = '8592050' # 'Lausanne, gare'
#stop_id_b = '8591818' # 'Lausanne-Flon, pl. de l'Europe'
stop_id_b = '8501210' # Bourdonnette
#stop_id_b = '8592173' # 'Prilly, centre'
#stop_id_b = '8501214' # 'Ecublens VD, EPFL'

latest_arrival_time = 66180 # 18:23:00
k=3
results = find_routes(G, stop_id_a, stop_id_b, latest_arrival_time, k)
results

[(65460,
  [('8592050', 'Lausanne, gare', None, None, None, None),
   ('8591818',
    "Lausanne-Flon, pl. de l'Europe",
    65460,
    65580,
    'M',
    '91-m2-j24-1'),
   ('8592133', 'Lausanne, Vigie', 65700, 65760, 'M', '91-m1-j24-1'),
   ('8501207', 'Lausanne, Montelly', 65760, 65880, 'M', '91-m1-j24-1'),
   ('8501208', 'Lausanne, Provence', 65880, 65940, 'M', '91-m1-j24-1'),
   ('8501209', 'Lausanne, Malley', 65940, 66000, 'M', '91-m1-j24-1'),
   ('8501210', 'Lausanne, Bourdonnette', 66000, 66180, 'M', '91-m1-j24-1')]),
 (65100,
  [('8592050', 'Lausanne, gare', None, None, None, None),
   ('8591818',
    "Lausanne-Flon, pl. de l'Europe",
    65100,
    65160,
    'M',
    '91-m2-j24-1'),
   ('8590442',
    'Lausanne, Riponne-M. Béjart',
    65160,
    65220,
    'M',
    '91-m2-j24-1'),
   ('8591988', 'Lausanne, Bel-Air', 65340, 65460, 'B', '92-1-V-j24-1'),
   ('8592010', 'Lausanne, Chauderon', 65460, 65580, 'B', '92-9-P-j24-1'),
   ('8592122', 'Lausanne, Tivoli', 65580, 65700, '